In [ ]:
# Step 1: Import Necessary Libraries
import warnings
warnings.filterwarnings("ignore")
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from google.colab import files

# Enable GPU if available
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# -------------------------------------------------------------------------------------------------------------------------------

# Step 2: Define Paths & Parameters
BASE_PATH = "/content/snapshots"
MODEL_PATH = os.path.join(BASE_PATH, "models")
HISTORY_PATH = os.path.join(BASE_PATH, "histories")
PLOTS_PATH = os.path.join(BASE_PATH, "plots")

# Ensure necessary directories exist
os.makedirs(MODEL_PATH, exist_ok=True)
os.makedirs(HISTORY_PATH, exist_ok=True)
os.makedirs(PLOTS_PATH, exist_ok=True)

# Image Processing Parameters
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 64  # Bigger batch for better utilization
EPOCHS = 25  # Increased epochs for better training

# Save hyperparameters
hyperparams = {
    "image_size": IMAGE_SIZE,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS
}
with open(os.path.join(BASE_PATH, "hyperparams.json"), "w") as f:
    json.dump(hyperparams, f)

# -------------------------------------------------------------------------------------------------------------------------------

# Step 3: Load Dataset from npy Files
X_path = "/content/X.npy"
y_path = "/content/y.npy"

if os.path.exists(X_path) and os.path.exists(y_path):
    print("Loading dataset from provided .npy files...")
    X = np.load(X_path)
    y = np.load(y_path)
else:
    raise FileNotFoundError("Dataset files X.npy and y.npy not found in Colab. Please upload them manually.")

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------------------------------------------------------------------------------------------------------------------

# Step 4: Define Multiple CNN Models
def build_baseline_cnn():
    """Baseline CNN Model (Original)"""
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 3)),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(1, activation="sigmoid")  # Binary classification
    ])
    return model

def build_vgg_like():
    """Deeper VGG-like CNN"""
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=(128, 128, 3)),
        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
        layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])
    return model

def build_resnet_like():
    """ResNet-inspired CNN with residual connections"""
    inputs = keras.Input(shape=(128, 128, 3))

    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(inputs)
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    res = layers.Conv2D(64, (1, 1), activation="relu", padding="same")(inputs)
    x = layers.add([x, res])
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    return keras.Model(inputs, outputs)

def build_mobilenet():
    """Lightweight MobileNet-based model"""
    base_model = keras.applications.MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights=None)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(128, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(base_model.input, outputs)

# Model dictionary
models_dict = {
    "baseline_cnn": build_baseline_cnn,
    "vgg_like": build_vgg_like,
    "resnet_like": build_resnet_like,
    "mobilenet": build_mobilenet
}

# -------------------------------------------------------------------------------------------------------------------------------

# Step 5: Train and Save Models
def train_and_save_model(model_name, model_fn):
    model_path = os.path.join(MODEL_PATH, f"{model_name}.h5")
    history_path = os.path.join(HISTORY_PATH, f"{model_name}.json")
    plot_path = os.path.join(PLOTS_PATH, f"{model_name}.png")

    if os.path.exists(model_path):
        print(f"Model {model_name} already exists. Skipping training.")
    else:
        print(f"Training {model_name}...")
        with tf.device('/GPU:0'):
            model = model_fn()
            model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
            history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, batch_size=BATCH_SIZE)

        model.save(model_path)
        with open(history_path, "w") as f:
            json.dump(history.history, f)

        plt.figure()
        plt.plot(history.history["accuracy"], label="Train Accuracy")
        plt.plot(history.history["val_accuracy"], label="Val Accuracy")
        plt.legend()
        plt.title(f"{model_name} Accuracy")
        plt.savefig(plot_path)
        plt.close()

for name, fn in models_dict.items():
    train_and_save_model(name, fn)

# -------------------------------------------------------------------------------------------------------------------------------

# Step 6: Download All Files
print("\nDownloading all models and training logs...")
for folder in [MODEL_PATH, HISTORY_PATH, PLOTS_PATH]:
    for file in os.listdir(folder):
        files.download(os.path.join(folder, file))


GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Loading dataset from provided .npy files...
Training baseline_cnn...
Epoch 1/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 109ms/step - accuracy: 0.7389 - loss: 0.5155 - val_accuracy: 0.9096 - val_loss: 0.2345
Epoch 2/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.9239 - loss: 0.2006 - val_accuracy: 0.9255 - val_loss: 0.1886
Epoch 3/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.9494 - loss: 0.1454 - val_accuracy: 0.9194 - val_loss: 0.2120
Epoch 4/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.9514 - loss: 0.1367 - val_accuracy: 0.9463 - val_loss: 0.1462
Epoch 5/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.9553 - loss: 0.1176 - val_accuracy: 0.9585 - val_loss: 0.1170
Epoch 6/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.9651 - loss: 0.0788 - val_accuracy: 0.9536 - val_loss: 0.1296
Epoch 7/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9782 - loss: 0.0570 

Training vgg_like...
Epoch 1/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 44s 475ms/step - accuracy: 0.6439 - loss: 0.6670 - val_accuracy: 0.8718 - val_loss: 0.3030
Epoch 2/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 48s 159ms/step - accuracy: 0.8748 - loss: 0.3178 - val_accuracy: 0.9219 - val_loss: 0.2196
Epoch 3/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 159ms/step - accuracy: 0.9241 - loss: 0.2151 - val_accuracy: 0.9267 - val_loss: 0.1892
Epoch 4/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 8s 161ms/step - accuracy: 0.9278 - loss: 0.1802 - val_accuracy: 0.9304 - val_loss: 0.1811
Epoch 5/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 161ms/step - accuracy: 0.9403 - loss: 0.1679 - val_accuracy: 0.9451 - val_loss: 0.1548
Epoch 6/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 161ms/step - accuracy: 0.9418 - loss: 0.1517 - val_accuracy: 0.8913 - val_loss: 0.2739
Epoch 7/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 161ms/step - accuracy: 0.9280 - loss: 0.1835 - val_accuracy: 0.9402 - val_loss: 0.1613
Epoch 8/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 8s 160ms/step - accuracy: 0.9530 - loss

Training resnet_like...
Epoch 1/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 15s 201ms/step - accuracy: 0.6219 - loss: 1.8248 - val_accuracy: 0.8889 - val_loss: 0.2667
Epoch 2/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 6s 120ms/step - accuracy: 0.9050 - loss: 0.2346 - val_accuracy: 0.8987 - val_loss: 0.2330
Epoch 3/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 120ms/step - accuracy: 0.9378 - loss: 0.1710 - val_accuracy: 0.8620 - val_loss: 0.3151
Epoch 4/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 125ms/step - accuracy: 0.9357 - loss: 0.1733 - val_accuracy: 0.9255 - val_loss: 0.1998
Epoch 5/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 123ms/step - accuracy: 0.9694 - loss: 0.0928 - val_accuracy: 0.9524 - val_loss: 0.1522
Epoch 6/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 120ms/step - accuracy: 0.9727 - loss: 0.0682 - val_accuracy: 0.9487 - val_loss: 0.1510
Epoch 7/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 124ms/step - accuracy: 0.9831 - loss: 0.0414 - val_accuracy: 0.9548 - val_loss: 0.1515
Epoch 8/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 6s 120ms/step - accuracy: 0.9859 - l

Training mobilenet...
Epoch 1/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 82s 613ms/step - accuracy: 0.7164 - loss: 0.6041 - val_accuracy: 0.4383 - val_loss: 0.6950
Epoch 2/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 32s 81ms/step - accuracy: 0.9172 - loss: 0.2098 - val_accuracy: 0.4383 - val_loss: 0.7109
Epoch 3/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.9585 - loss: 0.1199 - val_accuracy: 0.4383 - val_loss: 0.7178
Epoch 4/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.9671 - loss: 0.0928 - val_accuracy: 0.4383 - val_loss: 0.7320
Epoch 5/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.9759 - loss: 0.0805 - val_accuracy: 0.4383 - val_loss: 0.8130
Epoch 6/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.9726 - loss: 0.0777 - val_accuracy: 0.4383 - val_loss: 0.8645
Epoch 7/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.9780 - loss: 0.0653 - val_accuracy: 0.4383 - val_loss: 1.0567
Epoch 8/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - accuracy: 0.9829 - loss: 0.0502 -

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>